In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.geocoders import ArcGIS
import re
import folium
from tqdm import tqdm
import time  # Para controlar requisições
import ipywidgets

tqdm.pandas(dynamic_ncols=True)

In [2]:
# Importação do arquivo principal
df = pd.read_csv("database/dadostratados/df_final.csv")

In [3]:

# Obtendo as coordenada

# Utilizando o geocodificador da ArcGIS
geolocator = ArcGIS(timeout=10)

def obter_coordenadas(endereco):
    try:
        location = geolocator.geocode(endereco)
        time.sleep(1)  # respeita limite de 1 requisição por segundo
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except:
        return pd.Series([None, None])

# Aplica a função com barra de progresso
df[['LAT', 'LNG']] = df['ENDERECO'].progress_apply(obter_coordenadas)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 881/881 [16:49<00:00,  1.15s/it]


In [4]:
df.to_csv("database/dadosTratados/df_final_comGeo.csv")

In [5]:
df

,Unnamed: 0,NOME,TIPO,ENDERECO,LOCAL,PONTUACAO,NUM_COMENTARIO,TEMPO_MIN_M,TEMPO_MAX_M,TEMPO_MIN,TEMPO_MAX,BAIRRO,LAT,LNG
0,0,Shopping RioMar Fortaleza,cafeteria,"R. Des. Lauro Nogueira, 1500 - Papicu, Fortale...","Shopping center em Fortaleza, Ceará",4.7,52356,45.0,180.0,0 days 00:45:00,0 days 03:00:00,Papicu,-3.742329,-38.471450
1,1,Greta Café,cafeteria,"Av. Antônio Sales, 2960 - Dionísio Torres, For...",$$,4.7,1094,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Dionísio Torres,-3.747024,-38.497650
2,2,Bardo Café,cafeteria,"R. Amélia Benebien, 471 - Papicu, Fortaleza - ...",Cafeteria,4.9,22,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Papicu,-3.737813,-38.476876
3,3,"Go Coffee: Cafés, Chás, Frappes, Chocolate Que...",cafeteria,"R. Pereira Valente, 578 - Meireles, Fortaleza ...",Cafeteria,4.6,62,10.0,10.0,0 days 00:10:00,0 days 00:10:00,Meireles,-3.732479,-38.497050
4,4,Le Pain Le Café | Aldeota,cafeteria,"Rua Professor Dias da Rocha, 670 - Meireles, F...",$$$,4.4,1277,20.0,90.0,0 days 00:20:00,0 days 01:30:00,Meireles,-3.735362,-38.491679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,876,Bolo,restaurante,"Santa rosa - Santa Rosa, Fortaleza - CE, 60703...",NaN,0.0,0,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Santa Rosa,-3.819936,-38.595282
877,877,Restaurante,restaurante,"R. Gama, 170 - Vila Velha, Fortaleza - CE, 603...",Restaurante,3.0,1,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Vila Velha,-3.712739,-38.600711
878,878,Restaurante,restaurante,"Rua Ministro Antônio Coelho - Vila Velha, Fort...",NaN,0.0,0,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Vila Velha,-3.714419,-38.592349
879,879,Pastel Quente,restaurante,"Av. Duque de Caxias, 235 - Centro, Fortaleza -...",Pastelaria brasileira,4.6,21,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Centro,-3.732958,-38.526787
